In [64]:
!pip install webdriver_manager

In [65]:
import os
import re
import time
import numpy as np
import pandas as pd
import gurobipy as gp
import sys
from gurobipy import GRB
from datetime import datetime
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [66]:
parameters = {
    'name': 'ElectricityArbitrage',
    'generator_name': 'ADK HUDSON___FALLS',
    'start_date': None,
    'num_periods': 24,
    'num_markets': 1,
    'num_batteries': 8,
    'battery_capacity': 100,
    'charge_loss': 0.95,
    'max_charge': 50,
    'max_discharge': 100
}

# OPTIGUIDE DATA CODE GOES HERE

In [67]:
def extract_date(file_path):
    match = re.match(r'^(\d+)', file_path.split('/')[4])

    return match.group(1)

In [68]:
def get_prices(start_date, generator_name):
    if start_date == None:
        start_date = datetime.today()
        start_date = start_date.strftime("%Y%m%d")

    dates = [start_date]

    driver_path = ChromeDriverManager().install()

    home_dir = os.path.expanduser("~")

    download_directory = os.path.join(home_dir, 'Downloads_CSV')
    chrome_options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': download_directory}
    chrome_options.add_experimental_option('prefs', prefs)

    # Initialize the driver, use try except blocks depending on version of selenium installed
    try:
        # Older selenium version that takes executable path as argument
        driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)
    except:
        try:
            # For selenium version 3.141.0 or above
            driver=webdriver.Chrome(options=chrome_options)
        except:
            sys.exit("Please install correct version of selenium")

    driver.get('http://mis.nyiso.com/public/P-24Blist.htm')
    all_links = driver.find_elements("xpath", "//a[@href]")

    if not os.path.exists(download_directory):
        os.makedirs(download_directory)

    for date in dates:
        for link in all_links:
            href = link.get_attribute('href')

            if ('csv' in href) and (date in href):
                link.click()
                time.sleep(2)
                break

    csv_files = [file for file in os.listdir(download_directory) if file.endswith('.csv')]
    prices_by_date = {}

    for file_name in csv_files:
        file_path = os.path.join(download_directory, file_name)
        date = extract_date(file_path)
        df = pd.read_csv(file_path)
        prices = df[df['Name'] == generator_name][['LBMP ($/MWHr)']].values.tolist()

        prices_by_date[date] = prices

    driver.quit()

    return prices_by_date

In [69]:
def create_model(parameters): # num_markets is going to be 1 for the time being
    name = parameters['name']
    generator_name = parameters['generator_name']
    start_date = parameters['start_date']
    num_periods = parameters['num_periods']
    num_markets = parameters['num_markets']
    num_batteries = parameters['num_batteries']
    battery_capacity = parameters['battery_capacity']
    charge_loss = parameters['charge_loss']
    max_charge = parameters['max_charge']
    max_discharge = parameters['max_discharge']

    model = gp.Model(name)

    periods = range(num_periods)
    markets = range(num_markets)

    buy = model.addVars(num_periods, num_markets, vtype=GRB.CONTINUOUS, name='Buy')
    sell = model.addVars(num_periods, num_markets, vtype=GRB.CONTINUOUS, name='Sell')

    prices_by_date = get_prices(start_date, generator_name)

    # placeholder
    start_date = datetime.today()
    start_date = start_date.strftime("%Y%m%d")

    print(prices_by_date)
    print(start_date)

    prices = prices_by_date[start_date]

    model.setObjective(
        gp.quicksum(prices[p][i] * sell[p, i] - prices[p][i] * buy[p, i] for p in periods for i in markets),
        GRB.MAXIMIZE
    )

    for p in range(num_periods):
        current_level = np.sum(charge_loss * buy[p_, i] - sell[p_, i] for p_ in range(p) for i in markets)

        # model.addConstr(current_level <= 0, f'EnoughToSellConstraint_period_{p+1}')

        for i in markets:
            model.addConstr(current_level <= battery_capacity * num_batteries, f'CapacityConstraint_period_{p+1}')
            model.addConstr(sell[p, i] <= current_level, f'SupplyConstraint_period_{p+1}')
            model.addConstr(buy[p, i] * charge_loss <= max_charge, f'ChargeConstraint_period_{p+1}')
            model.addConstr(sell[p, i] <= max_discharge, f'DischargeConstraint_period_{p+1}')

    # OPTIGUIDE CONSTRAINT CODE GOES HERE

    return [model, buy, sell]

In [70]:
def print_model(model, buy, sell, parameters):
    num_periods = parameters['num_periods']
    num_markets = parameters['num_markets']

    if model.status == GRB.OPTIMAL:
        print("\nOptimal Solution:")

        for p in range(num_periods):
            print(f"\nPeriod {p + 1}:")

            for i in range(num_markets):
                print(f"Buy from Market {i + 1}: {buy[p, i].x}")
                print(f"Sell to Market {i + 1}: {sell[p, i].x}")

        print(f"\nTotal Profit: {model.objVal}")
    else:
        print("No solution found")

In [71]:
def run(parameters):
    [model, buy, sell] = create_model(parameters)

    model.optimize()

    return [model, buy, sell]

    print_model(model, buy, sell, parameters)

In [72]:
run(parameters)

{'20231128': [[31.24], [32.35], [29.64], [30.23], [26.03], [25.12], [25.12], [26.85], [27.26], [28.35], [31.7], [27.24], [30.8], [31.21], [28.71], [31.79], [28.83], [31.79], [34.98], [35.39], [41.3], [42.82], [42.82], [51.03], [41.22], [35.42], [40.77], [40.79], [34.99], [35.0], [34.25], [33.33], [35.39], [35.39], [35.38], [43.53], [36.32], [36.31], [33.32], [33.58], [34.14], [34.16], [34.16], [31.21], [36.21], [37.82], [37.83], [34.13], [32.06], [35.92], [31.58], [31.74], [40.7], [31.72], [23.92], [29.85], [30.74], [30.76], [31.08], [31.92], [27.22], [27.28], [30.79], [32.67], [31.74], [31.23], [26.64], [26.64], [26.69], [29.61], [29.28], [26.9], [44.94], [32.33], [31.88], [28.08], [29.13], [29.71], [32.45], [32.87], [33.49], [66.69], [49.87], [35.98], [29.74], [28.07], [36.01], [40.73], [36.97], [30.37], [29.84], [27.96], [27.96], [31.9], [35.02], [27.21], [27.1], [26.48], [27.71], [27.55], [27.09], [26.41], [26.36], [26.0], [25.98], [25.26], [25.01], [24.3], [24.2], [24.23], [24.17]

/var/folders/np/9tdxr2056n39qzb1mpdsjrtc0000gn/T/ipykernel_33299/2672680832.py:38: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  current_level = np.sum(charge_loss * buy[p_, i] - sell[p_, i] for p_ in range(p) for i in markets)
